In [1]:
from selenium import webdriver
import subprocess
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
    
from selenium.webdriver.common.keys import Keys

import time
import datetime as dt
import schedule

import pandas as pd
import numpy as np
from pprint import pprint
import urllib.request
import json
import glob
import sys
import os

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

pd.options.display.float_format = '{:.2f}'.format

class NaverDataLabOpenAPI():
    """
    네이버 데이터랩 오픈 API 컨트롤러 클래스
    """

    def __init__(self, client_id, client_secret):
        """
        인증키 설정 및 검색어 그룹 초기화
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.keywordGroups = []
        self.url = "https://openapi.naver.com/v1/datalab/search"

    def add_keyword_groups(self, group_dict):
        """
        검색어 그룹 추가
        """

        keyword_gorup = {
            'groupName': group_dict['groupName'],
            'keywords': group_dict['keywords']
        }
        
        self.keywordGroups.append(keyword_gorup)
#         print(f">>> Num of keywordGroups: {len(self.keywordGroups)}")
        
    def get_data(self, startDate, endDate, timeUnit, device, ages, gender):
        """
        요청 결과 반환
        timeUnit - 'date', 'week', 'month'
        device - None, 'pc', 'mo'
        ages = [], ['1' ~ '11']
        gender = None, 'm', 'f'
        """

        # Request body
        body = json.dumps({
            "startDate": startDate,
            "endDate": endDate,
            "timeUnit": timeUnit,
            "keywordGroups": self.keywordGroups,
            "device": device,
            "ages": ages,
            "gender": gender
        }, ensure_ascii=False)
        
        # Results
        request = urllib.request.Request(self.url)
        request.add_header("X-Naver-Client-Id",self.client_id)
        request.add_header("X-Naver-Client-Secret",self.client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
#             Json Result
            result = json.loads(response.read())
            
            period = []
            ratio = []
            title = []
            pprint(result)
            if result['results'][0]['data']:
                for val in result['results']:
                    period.append(val['data'][-1]['period'])
                    ratio.append(val['data'][-1]['ratio'])
                    title.append(val['title'])

            df = pd.DataFrame({"title" : title, 'period' : period, "ratio" : ratio})
            
        else:
            print("Error Code:" + rescode)
            
        return df

keyword_group_set = {
    'keyword_group_1': {'groupName': "애플", 'keywords': ["애플"]},
}
def run():
    print("시작시작시작")
    try:
        subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp1"')  # 디버거 크롬 구동
    except:
        subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp1"')  # 디버거 크롬 구동
    option = Options()
    option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    # option.add_argument("--headless")

    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
    try:
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
    except:
        chromedriver_autoinstaller.install('./')
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)

    #####크롤링
    
    driver.get('https://blackkiwi.net/service/keyword-analysis?keyword=%EC%A0%9C%EB%A1%9C%EC%9B%A8%EC%9D%B4%EC%8A%A4%ED%8A%B8&platform=naver')
    time.sleep(10) #혹시 모를 에러 방지를 위한 적절한 wait 넣어주기
    title=driver.find_elements(By.CSS_SELECTOR,'table a')
    title_l=[]
    for i in range(20):
        title_l.append(title[i].text)
        print(title_l[i])
    driver.implicitly_wait(10)
    driver.get('http://surffing.net/MainSeoSearch.do')
    search=driver.find_element(By.ID,'saerchKeyword')
    search.send_keys('제로웨이스트')
    search.send_keys(Keys.ENTER)
    title2=driver.find_elements(By.CSS_SELECTOR,'tr>td.center:nth-child(1)')
    title_b=[]

    for i in range(len(title2)):
        title_b.append(title2[i].text)
        print(title_b[i])
    
    title_l.extend(title_b)
    list_a=title_l
    
    ##날짜지정
    today = dt.date.today()
    monday=today - dt.timedelta(days=today.weekday())
    end=monday.strftime('%Y-%m-%d')
    print(end)
    th=dt.timedelta(days=30)
    start=monday-th
    start=start.strftime('%Y-%m-%d')
        
    #####크롤링end
    cnt=0
    while(True):
        if len(list_a)>0 :
            for idx, (key, val) in enumerate(keyword_group_set.items()):
                val['groupName'] = list_a[idx]
                val['keywords'] = [list_a[idx]]
            del list_a[:1]
            cnt+=1
        else:
            break
    #     pprint(keyword_group_set)

            # API 인증 정보 설정
        client_id = "xTsOEpqU869lYEaqyaU6"
        client_secret = "kkp_ZP2j9T"

        # 요청 파라미터 설정
        startDate = start
        endDate = end
        timeUnit = 'week'
        device = ''
        ages = []
        gender = ''

        # 데이터 프레임 정의
        naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)

        naver.add_keyword_groups(keyword_group_set['keyword_group_1'])

        df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)

        if(cnt>1):
            df1=pd.concat([df1,df])
        else:
            df1=df
#     print(df1)
    driver.close()
    #검색주간과 같은 데이터만 추출
    df1=df1[df1['period']==end]
    #ratio높은순으로 정렬
    df2=df1.sort_values('ratio',ascending=False)
    #ratio 삭제
    df2=df2.drop([df2.columns[2]],axis=1)
    #인덱싱해주기
    #인덱싱해주기
    df2.reset_index(drop=True,inplace=True)
    #10위까지 저장
    data=df2.loc[:9]
    print(data)
    return data

# step3.실행 주기 설정
# schedule.every().monday.at("02:00").do(run)
schedule.every().day.at("17:51").do(run)
# step4.스캐쥴 시작
while True:
    schedule.run_pending()
    time.sleep(1)

시작시작시작
제로웨이스트마켓
제로 웨이스트비누
제로 웨이스트 챌린지
제로웨이스트 수세미
부산 제로웨이스트
대구 제로웨이스트
제로웨이스트 실천
캘러웨이 로스트볼
제로웨이스트 숙소
제로웨이스트 뜻
제로웨이스트 키트
제로웨이스트샵
행주
커플 팔찌
탄소중립
esg 경영
수세미
후드티
설거지
ESG
제로웨이스트
친환경신발
제주도어린이체험
업사이클링제품
친환경제품
친환경의류
업사이클링브랜드
친환경운동화
제주도수학여행
페트병재활용
지구의날행사
르무통
업사이클링
친환경브랜드
리사이클링
업사이클링가방
2022-08-22
{'endDate': '2022-08-26',
 'results': [{'data': [{'period': '2022-08-01', 'ratio': 100}],
              'keywords': ['제로웨이스트마켓'],
              'title': '제로웨이스트마켓'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-26',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 81.08108},
                       {'period': '2022-07-25', 'ratio': 97.29729},
                       {'period': '2022-08-01', 'ratio': 67.56756},
                       {'period': '2022-08-08', 'ratio': 40.54054},
                       {'period': '2022-08-15', 'ratio': 100},
                       {'period': '2022-08-22', 'ratio': 89.18918}],
              'keywords': ['제로 웨이스트비누'],
              'title': '제로 웨

{'endDate': '2022-08-26',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 91.97827},
                       {'period': '2022-07-25', 'ratio': 82.61217},
                       {'period': '2022-08-01', 'ratio': 81.02742},
                       {'period': '2022-08-08', 'ratio': 93.23361},
                       {'period': '2022-08-15', 'ratio': 100},
                       {'period': '2022-08-22', 'ratio': 63.10541}],
              'keywords': ['수세미'],
              'title': '수세미'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-26',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 41.70162},
                       {'period': '2022-07-25', 'ratio': 46.02908},
                       {'period': '2022-08-01', 'ratio': 55.25027},
                       {'period': '2022-08-08', 'ratio': 71.94491},
                       {'period': '2022-08-15', 'ratio': 100},
                       {'period': '2022-08-22', 'ratio': 81.07522}],
              'keywords'

{'endDate': '2022-08-26',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 95.24838},
                       {'period': '2022-07-25', 'ratio': 72.85817},
                       {'period': '2022-08-01', 'ratio': 74.87401},
                       {'period': '2022-08-08', 'ratio': 84.59323},
                       {'period': '2022-08-15', 'ratio': 100},
                       {'period': '2022-08-22', 'ratio': 81.35349}],
              'keywords': ['업사이클링'],
              'title': '업사이클링'}],
 'startDate': '2022-07-18',
 'timeUnit': 'week'}
{'endDate': '2022-08-26',
 'results': [{'data': [{'period': '2022-07-18', 'ratio': 79.93311},
                       {'period': '2022-07-25', 'ratio': 91.97324},
                       {'period': '2022-08-01', 'ratio': 80.93645},
                       {'period': '2022-08-08', 'ratio': 77.92642},
                       {'period': '2022-08-15', 'ratio': 82.27424},
                       {'period': '2022-08-22', 'ratio': 100}],
              'keywo

KeyboardInterrupt: 